In [17]:
import pandas as pd
import numpy as np
import pymannkendall as mk
import os
from typing import Dict, List, Tuple
import glob

# ======================
# DATA PROCESSING MODULE
# ======================

def load_holaps_data(file_path: str) -> pd.DataFrame:
    """
    Load HOLAPS model data and prepare for analysis
    
    Args:
        file_path: Path to HOLAPS CSV file
    
    Returns:
        Processed DataFrame with additional columns
    """
    df = pd.read_csv(file_path)
    
    # Convert temperature from Kelvin to Celsius if needed
    temp_cols = ['Ta', 'Td', 'Ts']
    for col in temp_cols:
        if col in df.columns:
            # Assuming temperatures are in Kelvin, convert to Celsius
            df[col] = df[col] - 273.15
    
    # Create datetime column for easier time-based operations
    df['date'] = pd.to_datetime(df[['year', 'month']].assign(day=1))
    df['Year'] = df['year']
    df['Month'] = df['month']
    
    return df

# ======================
# COMPLETENESS ANALYSIS
# ======================

def validate_holaps_variable(df: pd.DataFrame, var: str) -> List[int]:
    """
    Identify complete years for a HOLAPS variable
    
    Args:
        df: Input DataFrame with data
        var: Variable name to validate
    
    Returns:
        List of years with complete data (12 months)
    """
    # HOLAPS data typically doesn't have QC flags, just check for non-NaN values
    valid_mask = df[var].notna()
    
    valid_data = df[valid_mask].copy()
    valid_year_counts = valid_data['Year'].value_counts()
    return valid_year_counts[valid_year_counts == 12].index.tolist()

def get_complete_years_holaps(df: pd.DataFrame, variables: List[str]) -> List[int]:
    """
    Find years where all specified variables have complete data
    
    Args:
        df: Input DataFrame with data
        variables: List of variables to check
    
    Returns:
        Sorted list of years with complete data for all variables
    """
    common_years = set(df['Year'].unique())
    for var in variables:
        valid_years = validate_holaps_variable(df, var)
        common_years.intersection_update(valid_years)
    return sorted(common_years)

def save_holaps_variable_availability(df: pd.DataFrame, variables: List[str], output_file: str) -> None:
    """
    Save availability of each variable (years with valid data)
    
    Args:
        df: DataFrame with Year and variable columns
        variables: List of variables to check
        output_file: Path to save CSV
    """
    availability_records = []
    
    for var in variables:
        valid_years = validate_holaps_variable(df, var)
        availability_records.append({
            "Variable": var,
            "Valid_Years": ", ".join(map(str, sorted(valid_years))),
            "Count_Valid_Years": len(valid_years)
        })
    
    availability_df = pd.DataFrame(availability_records)
    availability_df.to_csv(output_file, index=False)
    print(f"HOLAPS variable availability saved to {output_file}")

# ======================
# YEARLY ANALYSIS
# ======================

def calculate_holaps_yearly_means(df: pd.DataFrame, variables: List[str], years: List[int]) -> pd.DataFrame:
    """
    Calculate yearly means for complete years only
    
    Args:
        df: Input DataFrame with data
        variables: Variables to calculate means for
        years: Years to include
    
    Returns:
        DataFrame with yearly means
    """
    if not years:
        return pd.DataFrame()
    
    complete_data = df[df['Year'].isin(years)]
    yearly_means = complete_data.groupby('Year')[variables].mean()
    return yearly_means.reset_index()

def generate_holaps_yearly_analysis(df: pd.DataFrame, variables: List[str]) -> Dict[str, pd.DataFrame]:
    """
    Generate yearly means for HOLAPS data
    
    Args:
        df: Input DataFrame with data
        variables: List of variables to analyze
    
    Returns:
        Dictionary of DataFrames with yearly means
    """
    results = {}
    
    # Get complete years for all variables
    complete_years = get_complete_years_holaps(df, variables)
    
    if complete_years:
        # Calculate yearly means for complete period
        yearly_means = calculate_holaps_yearly_means(df, variables, complete_years)
        results["Complete Period"] = yearly_means
        
        # Also calculate for all available years (individual variable completeness)
        for var in variables:
            var_years = validate_holaps_variable(df, [var])
            if var_years:
                var_means = calculate_holaps_yearly_means(df, [var], var_years)
                results[f"Single Variable - {var}"] = var_means
    
    return results

# ======================
# SEASONAL ANALYSIS
# ======================

def get_season_months(season: str) -> List[int]:
    """Return months for each season (with DJF year adjustment)"""
    return {
        'DJF': [12, 1, 2],  # December, January, February
        'MAM': [3, 4, 5],   # March, April, May
        'JJA': [6, 7, 8],   # June, July, August
        'SON': [9, 10, 11]  # September, October, November
    }[season]

def get_complete_seasons_holaps(df: pd.DataFrame, variables: List[str], season: str) -> List[int]:
    """
    Get seasons with complete data for all variables
    
    Args:
        df: Input DataFrame
        variables: Variables to check
        season: Season code (DJF, MAM, JJA, SON)
    
    Returns:
        List of complete season years
    """
    season_months = get_season_months(season)
    df_season = df[df['Month'].isin(season_months)].copy()
    
    # Adjust year for winter season (DJF)
    if season == 'DJF':
        df_season['Season_Year'] = df_season['Year']
        df_season.loc[df_season['Month'] == 12, 'Season_Year'] += 1
    else:
        df_season['Season_Year'] = df_season['Year']
    
    # Find seasons with all months present
    month_counts = df_season.groupby(['Season_Year'])['Month'].nunique()
    complete_seasons = month_counts[month_counts == 3].index.tolist()
    
    # Check variable completeness
    valid_seasons = []
    for season_year in complete_seasons:
        season_data = df_season[df_season['Season_Year'] == season_year]
        valid = True
        for var in variables:
            var_data = season_data[season_data[var].notna()]
            if len(var_data) < 3:  # Not all months valid
                valid = False
                break
        if valid:
            valid_seasons.append(season_year)
    
    return sorted(valid_seasons)

def calculate_holaps_seasonal_means(df: pd.DataFrame, variables: List[str], season: str, season_years: List[int]) -> pd.DataFrame:
    """
    Calculate seasonal means for complete seasons
    
    Args:
        df: Input DataFrame
        variables: Variables to calculate means for
        season: Season code (DJF, MAM, JJA, SON)
        season_years: List of years with complete seasonal data
    
    Returns:
        DataFrame with seasonal means
    """
    if not season_years:
        return pd.DataFrame()
    
    season_months = get_season_months(season)
    df_season = df[df['Month'].isin(season_months)].copy()
    
    # Adjust year for winter season (DJF)
    if season == 'DJF':
        df_season['Season_Year'] = df_season['Year']
        df_season.loc[df_season['Month'] == 12, 'Season_Year'] += 1
    else:
        df_season['Season_Year'] = df_season['Year']
    
    # Filter for complete seasons
    df_season = df_season[df_season['Season_Year'].isin(season_years)]
    
    # Calculate seasonal means
    seasonal_means = df_season.groupby('Season_Year')[variables].mean()
    seasonal_means['Season'] = season
    return seasonal_means.reset_index()

def generate_holaps_seasonal_analysis(df: pd.DataFrame, variables: List[str], season: str) -> Dict[str, pd.DataFrame]:
    """
    Generate seasonal means for HOLAPS data
    
    Args:
        df: Input DataFrame
        variables: Variables to analyze
        season: Season code (DJF, MAM, JJA, SON)
    
    Returns:
        Dictionary of DataFrames with seasonal means
    """
    results = {}
    
    # Get complete seasons for all variables
    complete_seasons = get_complete_seasons_holaps(df, variables, season)
    
    if complete_seasons:
        # Calculate seasonal means for complete period
        seasonal_means = calculate_holaps_seasonal_means(df, variables, season, complete_seasons)
        results["Complete Period"] = seasonal_means
        
        # Also calculate for all available seasons (individual variable completeness)
        for var in variables:
            # For single variable, we can be less strict about completeness
            var_season_data = df[df['Month'].isin(get_season_months(season))].copy()
            
            # Adjust year for winter season
            if season == 'DJF':
                var_season_data['Season_Year'] = var_season_data['Year']
                var_season_data.loc[var_season_data['Month'] == 12, 'Season_Year'] += 1
            else:
                var_season_data['Season_Year'] = var_season_data['Year']
            
            # Filter for seasons with this variable available
            var_valid = var_season_data[var_season_data[var].notna()]
            var_season_counts = var_valid.groupby('Season_Year').size()
            var_complete_seasons = var_season_counts[var_season_counts == 3].index.tolist()
            
            if var_complete_seasons:
                var_means = calculate_holaps_seasonal_means(df, [var], season, var_complete_seasons)
                results[f"Single Variable - {var}"] = var_means
    
    return results

# ======================
# TREND ANALYSIS
# ======================

def calculate_holaps_trend_stats(series: pd.Series) -> Dict[str, float]:
    """
    Calculate Mann-Kendall trend statistics for HOLAPS data
    
    Args:
        series: Time series data with Year as index
    
    Returns:
        Dictionary with slope, p-value, and period info
    """
    if series.notna().sum() < 10:
        return None
    
    start_year = int(series.index.min())
    end_year = int(series.index.max())
    full_index = np.arange(start_year, end_year + 1, 1)
    series = series.reindex(full_index)
    
    try:
        result = mk.original_test(series.values)
        return {
            'slope': result.slope,
            'p_value': result.p,
            'period': f"{start_year}-{end_year} ({series.notna().sum()})"
        }
    except Exception:
        return None

def build_holaps_trend_tables(sheets: Dict[str, pd.DataFrame]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Build p-value and slope tables from HOLAPS analysis results
    
    Args:
        sheets: Dictionary of DataFrames from analysis
    
    Returns:
        Tuple of (p-value table, slope table)
    """
    p_table_data = []
    slope_table_data = []
    
    for sheet_name, data in sheets.items():
        if data.empty:
            continue
            
        # Set appropriate index
        if 'Year' in data.columns:
            data_indexed = data.set_index('Year')
        elif 'Season_Year' in data.columns:
            data_indexed = data.set_index('Season_Year')
        else:
            continue
        
        # Get variables (exclude metadata columns)
        vars_included = [col for col in data_indexed.columns if col not in ['Season', 'station_id', 'latitude', 'longitude']]
        
        # Calculate stats for all variables
        stats_dict = {}
        period = None
        
        for var in vars_included:
            stats = calculate_holaps_trend_stats(data_indexed[var].dropna())
            if stats:
                stats_dict[var] = {
                    'p_value': stats['p_value'],
                    'slope': stats['slope']
                }
                period = stats['period']
        
        if stats_dict:
            p_row = {'Analysis': sheet_name, 'Period': period}
            s_row = {'Analysis': sheet_name, 'Period': period}
            
            for var in vars_included:
                if var in stats_dict:
                    p_row[var] = stats_dict[var]['p_value']
                    s_row[var] = stats_dict[var]['slope']
                else:
                    p_row[var] = np.nan
                    s_row[var] = np.nan
            
            p_table_data.append(p_row)
            slope_table_data.append(s_row)
    
    return pd.DataFrame(p_table_data), pd.DataFrame(slope_table_data)

# ======================
# MAIN EXECUTION
# ======================

def run_holaps_analysis(station_code: str):
    """
    Run complete HOLAPS analysis pipeline
    
    Args:
        station_code: Station code (e.g., "FR-Hes")
    """
    # Define HOLAPS variables
    holaps_variables = ['GHF', 'H', 'LE', 'Rn', 'Ta', 'Td', 'Ts', 'rain', 'sm1','sm2','sm3','sm4','sm5']
    
    # Build file path
    base_dir = r"c:\\Deepak\\HOLAPS\\monthly\\station_data2"
   
    holaps_file = os.path.join(base_dir, f"{station_code}_all_variables.csv")
    
    # Check if file exists
    if not os.path.exists(holaps_file):
        print(f"HOLAPS file not found for {station_code}: {holaps_file}")
        return
    
    # Create output directory
    output_dir = os.path.join(base_dir, station_code)
    os.makedirs(output_dir, exist_ok=True)
    
    # Load data
    print(f"Loading HOLAPS data for {station_code}...")
    df = load_holaps_data(holaps_file)
    
    # Basic info
    print(f"Data period: {df['Year'].min()} to {df['Year'].max()}")
    print(f"Total years: {df['Year'].nunique()}")
    
    # Save variable availability
    availability_file = os.path.join(output_dir, f"{station_code}_holaps_variable_availability.csv")
    save_holaps_variable_availability(df, holaps_variables, availability_file)
    
    # Yearly analysis
    print("Performing yearly analysis...")
    yearly_results = generate_holaps_yearly_analysis(df, holaps_variables)
    
    yearly_file = os.path.join(output_dir, f'holaps_yearly_means_{station_code}.xlsx')
    if yearly_results:
        with pd.ExcelWriter(yearly_file) as writer:
            for sheet_name, data in yearly_results.items():
                if not data.empty:
                    data.to_excel(writer, sheet_name=sheet_name[:31], index=False)
        print(f"Yearly means saved to {yearly_file}")
    else:
        print("No complete yearly data available")
    
    # Seasonal analysis
    print("Performing seasonal analysis...")
    seasons = ['DJF', 'MAM', 'JJA', 'SON']
    
    for season in seasons:
        seasonal_results = generate_holaps_seasonal_analysis(df, holaps_variables, season)
        seasonal_file = os.path.join(output_dir, f"holaps_seasonal_means_{station_code}_{season}.xlsx")
        
        if seasonal_results:
            with pd.ExcelWriter(seasonal_file) as writer:
                for sheet_name, data in seasonal_results.items():
                    if not data.empty:
                        data.to_excel(writer, sheet_name=sheet_name[:31], index=False)
            print(f"Seasonal means for {season} saved to {seasonal_file}")
        else:
            print(f"No complete seasonal data available for {season}")
    
    # Trend analysis - Yearly
    print("Performing trend analysis...")
    if os.path.exists(yearly_file):
        yearly_sheets = pd.read_excel(yearly_file, sheet_name=None)
        p_table, slope_table = build_holaps_trend_tables(yearly_sheets)
        
        trend_file = os.path.join(output_dir, f'holaps_trend_tables_{station_code}_yearly.xlsx')
        with pd.ExcelWriter(trend_file) as writer:
            p_table.to_excel(writer, sheet_name='P-values', index=False)
            slope_table.to_excel(writer, sheet_name='Slopes', index=False)
        print(f"Yearly trend tables saved to {trend_file}")
    
    # Trend analysis - Seasonal
    for season in seasons:
        seasonal_file = os.path.join(output_dir, f"holaps_seasonal_means_{station_code}_{season}.xlsx")
        if os.path.exists(seasonal_file):
            try:
                seasonal_sheets = pd.read_excel(seasonal_file, sheet_name=None)
                p_table, slope_table = build_holaps_trend_tables(seasonal_sheets)
                
                trend_file = os.path.join(output_dir, f'holaps_trend_tables_{station_code}_{season}.xlsx')
                with pd.ExcelWriter(trend_file) as writer:
                    p_table.to_excel(writer, sheet_name='P-values', index=False)
                    slope_table.to_excel(writer, sheet_name='Slopes', index=False)
                print(f"Seasonal trend tables for {season} saved to {trend_file}")
            except Exception as e:
                print(f"Error processing seasonal trends for {season}: {e}")

def run_all_holaps_stations():
    base_dir = r"c:\\Deepak\\HOLAPS\\monthly\\station_data2"
    
    # Changed pattern to find all_variables files
    holaps_files = glob.glob(os.path.join(base_dir, "*_all_variables.csv"))
    
    # Extract station codes from filenames like "BE-Bra_all_variables.csv"
    stations = []
    for file_path in holaps_files:
        filename = os.path.basename(file_path)
        station_code = filename.replace("_all_variables.csv", "")
        stations.append(station_code)
    
    print(f"Found {len(stations)} HOLAPS stations: {stations}")
    
    # Run analysis for each station
    for station in stations:
        print(f"\n{'='*50}")
        print(f"Processing HOLAPS station: {station}")
        print(f"{'='*50}")
        
        try:
            run_holaps_analysis(station)
            print(f"✓ Completed analysis for {station}")
        except Exception as e:
            print(f"✗ Error processing {station}: {e}")

# Run for specific station
if __name__ == "__main__":
    # Run for a single station
    # run_holaps_analysis("FR-Hes")
    
    # Or run for all available stations
    run_all_holaps_stations()

Found 47 HOLAPS stations: ['AT-Neu', 'BE-Bra', 'BE-Lon', 'BE-Vie', 'CH-Cha', 'CH-Dav', 'CH-Fru', 'CZ-BK1', 'CZ-wet', 'DE-Geb', 'DE-Gri', 'DE-Hai', 'DE-HoH', 'DE-Hzd', 'DE-Kli', 'DE-Lnf', 'DE-Obe', 'DE-RuR', 'DE-RuS', 'DE-Tha', 'DK-Sor', 'ES-Agu', 'ES-LJu', 'FI-Hyy', 'FI-Let', 'FR-Aur', 'FR-Bil', 'FR-Gri', 'FR-Hes', 'FR-Lam', 'FR-LBr', 'IT-BCi', 'IT-Col', 'IT-Cp2', 'IT-Cpz', 'IT-Lav', 'IT-MBo', 'IT-Ren', 'IT-Ro2', 'IT-SR2', 'IT-SRo', 'IT-Tor', 'IT-TrF', 'NL-Loo', 'RU-Fyo', 'SE-Htm', 'SE-Nor']

Processing HOLAPS station: AT-Neu
Loading HOLAPS data for AT-Neu...
Data period: 2001 to 2020
Total years: 20
HOLAPS variable availability saved to c:\\Deepak\\HOLAPS\\monthly\\station_data2\AT-Neu\AT-Neu_holaps_variable_availability.csv
Performing yearly analysis...
Yearly means saved to c:\\Deepak\\HOLAPS\\monthly\\station_data2\AT-Neu\holaps_yearly_means_AT-Neu.xlsx
Performing seasonal analysis...
Seasonal means for DJF saved to c:\\Deepak\\HOLAPS\\monthly\\station_data2\AT-Neu\holaps_seasonal_

KeyboardInterrupt: 

In [6]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 7822-CCB4

 Directory of c:\Deepak\HOLAPS\monthly\station_data2

10/22/2025  11:11 AM    <DIR>          .
10/21/2025  06:23 PM    <DIR>          ..
10/22/2025  11:07 AM    <DIR>          AT-Neu
10/21/2025  06:23 PM            81,298 AT-Neu_all_variables.csv
10/22/2025  11:07 AM    <DIR>          BE-Bra
10/21/2025  06:23 PM            80,459 BE-Bra_all_variables.csv
10/22/2025  11:07 AM    <DIR>          BE-Lon
10/21/2025  06:23 PM            81,526 BE-Lon_all_variables.csv
10/22/2025  11:07 AM    <DIR>          BE-Vie
10/21/2025  06:23 PM            80,880 BE-Vie_all_variables.csv
10/22/2025  11:07 AM    <DIR>          CH-Cha
10/21/2025  06:24 PM            80,372 CH-Cha_all_variables.csv
10/22/2025  11:07 AM    <DIR>          CH-Dav
10/21/2025  06:24 PM            80,457 CH-Dav_all_variables.csv
10/22/2025  11:07 AM    <DIR>          CH-Fru
10/21/2025  06:24 PM            81,218 CH-Fru_all_variables.csv
10/22/2025  11:07 AM    <DI

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.lines import Line2D

# Define HOLAPS parameters and assign fixed colors
color_dict = {
    'GHF': '#1f77b4',          # blue
    'H': '#ff7f0e',            # orange
    'LE': '#2ca02c',           # green
    'Rn': '#d62728',           # red
    'Ta': '#9467bd',           # purple
    'Td': '#8c564b',           # brown
    'Ts': '#e377c2',           # pink
    'rain': '#17becf',         # teal
    'sm1': '#bcbd22'           # olive
}

parameters = list(color_dict.keys())

# Fixed absolute Y positions for each parameter
parameter_distances = {
    'GHF': 0.2,
    'H': 0.4,
    'LE': 0.6,
    'Rn': 0.8,
    'Ta': 1.0,
    'Td': 1.2,
    'Ts': 1.4,
    'rain': 1.6,
    'sm1': 1.8
}

# Directory containing station subdirectories
input_directory = r'C:\Deepak\HOLAPS\monthly\station_data2'
output_directory = r'C:\Deepak\HOLAPS\monthly\station_data2\plots'

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Get list of all station subdirectories
station_dirs = [d for d in os.listdir(input_directory) 
                if os.path.isdir(os.path.join(input_directory, d))]

print(f"Found {len(station_dirs)} station directories")

# Process each station directory
for station_dir in station_dirs:
    try:
        station_path = os.path.join(input_directory, station_dir)
        
        # Look for yearly trend file in this station directory
        trend_files = [f for f in os.listdir(station_path) 
                      if f.endswith('_yearly.xlsx') and 'trend' in f.lower()]
        
        if not trend_files:
            print(f"No yearly trend file found for {station_dir}")
            continue
            
        trend_file = trend_files[0]
        station_name = station_dir  # Use directory name as station name
        
        print(f"Processing {station_name}...")
        
        # Create figure
        fig, ax = plt.subplots(1, 1, figsize=(20, 12))
        
        # File path
        file_path = os.path.join(station_path, trend_file)
        
        # Read the Excel file
        p_values_df = pd.read_excel(file_path, sheet_name='P-values')
        slopes_df = pd.read_excel(file_path, sheet_name='Slopes')
        
        print(f"  P-values shape: {p_values_df.shape}")
        print(f"  Slopes shape: {slopes_df.shape}")
        print(f"  P-values columns: {list(p_values_df.columns)}")
        print(f"  Slopes columns: {list(slopes_df.columns)}")
        
        # Clean the data
        p_values_df = p_values_df.dropna(how='all')
        slopes_df = slopes_df.dropna(how='all')
        p_values_df = p_values_df.reset_index(drop=True)
        slopes_df = slopes_df.reset_index(drop=True)
        
        # Get periods and analyses
        periods = slopes_df['Period'].values
        analyses = slopes_df['Analysis'].values
        
        print(f"  Periods: {periods}")
        print(f"  Analyses: {analyses}")
        
        # Plot data for each period/analysis
        for period_idx, (period, analysis) in enumerate(zip(periods, analyses)):
            for param in parameters:
                if param not in slopes_df.columns or param not in p_values_df.columns:
                    continue
                    
                slope = slopes_df[param].iloc[period_idx]
                p_val = p_values_df[param].iloc[period_idx]

                if pd.isna(p_val) or pd.isna(slope):
                    continue

                print(f"    {param}: slope={slope:.4f}, p-value={p_val:.4f}")

                # FIXED POSITION: Use the predefined distance from zero
                base_distance = parameter_distances[param]
                y_position = base_distance if slope >= 0 else -base_distance
                
                x_position = period_idx  # Center on the period position
                
                # Small random offset to avoid perfect alignment
                x_offset = np.random.uniform(-0.15, 0.15)
                x_position += x_offset

                # Marker rules based on p-values
                if p_val < 0.1:
                    marker = 'o'   # full circle
                    marker_size = 300
                    fill_style = 'full'
                    edge_width = 4.0
                    alpha_val = 1.0
                elif p_val < 0.2:
                    marker = '^'   # triangle
                    marker_size = 260
                    fill_style = 'full'
                    edge_width = 4.0
                    alpha_val = 0.9
                else:
                    marker = 'o'   # open circle
                    marker_size = 220
                    fill_style = 'none'
                    edge_width = 3.5
                    alpha_val = 0.8

                # Plot marker
                ax.scatter(x_position, y_position, s=marker_size,
                           c=color_dict[param] if fill_style == 'full' else 'none',
                           marker=marker, edgecolors=color_dict[param],
                           linewidths=edge_width, alpha=alpha_val, zorder=5)
        
        # Customize plot
        ax.set_xlabel('Analysis Periods', fontsize=20, fontweight='bold')
        ax.set_xticks(range(len(periods)))
        
        # Create labels showing both analysis type and period
        x_labels = [f"{analyses[i]}\n({periods[i]})" for i in range(len(periods))]
        ax.set_xticklabels(x_labels, rotation=45, ha='right', fontsize=16)
        
        # Set y-axis limits based on maximum distance
        max_distance = max(parameter_distances.values())
        y_limit = max_distance * 1.2  # Add some padding
        ax.set_ylim(-y_limit, y_limit)
        
        # Remove y-axis labels and ticks
        ax.set_yticks([])
        ax.set_ylabel('')
        
        # Zero line
        ax.axhline(y=0, color='black', linestyle='-', alpha=0.8, linewidth=3, zorder=1)
        
        # Customize spines
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_visible(False)

        # Add vertical lines to separate analysis groups
        for i in range(len(periods) + 1):
            ax.axvline(x=i - 0.5, color='gray', linestyle='--', alpha=0.5, linewidth=2, zorder=1)

        # Add grid for better readability
        ax.grid(True, axis='y', linestyle='--', alpha=0.4, linewidth=1.2, zorder=1)

        # Main title
        plt.title(f'HOLAPS Trend Analysis for {station_name}',
                 fontsize=22, fontweight='bold', pad=20)

        # Create legends
        legend_elements = [
            Line2D([0], [0], marker='s', color='w',
                   markerfacecolor=color_dict[param],
                   markersize=16, label=param)
            for param in parameters
        ]

        marker_elements = [
            Line2D([0], [0], marker='o', color='black', markerfacecolor='black',
                   markersize=20, linestyle='None', label='p < 0.1 (Full circle)'),
            Line2D([0], [0], marker='^', color='black', markerfacecolor='black',
                   markersize=20, linestyle='None', label='0.1 ≤ p < 0.2 (Triangle)'),
            Line2D([0], [0], marker='o', color='black', markerfacecolor='none',
                   markersize=20, linestyle='None', label='p ≥ 0.2 (Open circle)')
        ]

        # Place legends
        legend1 = ax.legend(handles=legend_elements, 
                          loc='upper left',
                          bbox_to_anchor=(1.02, 1),
                          fontsize=14,
                          title='HOLAPS Parameters', 
                          title_fontsize=16)

        legend2 = ax.legend(handles=marker_elements, 
                          loc='upper left',
                          bbox_to_anchor=(1.02, 0.7),
                          fontsize=14,
                          title='Significance Levels', 
                          title_fontsize=16)

        # Add both legends to the plot
        ax.add_artist(legend1)
        
        # Adjust layout for larger elements
        plt.tight_layout(rect=[0, 0, 0.82, 0.95])
        
        # Save the plot with high DPI
        output_path = os.path.join(output_directory, f"{station_name}_holaps_trend_plot.png")
        plt.savefig(output_path, dpi=350, bbox_inches='tight')
        plt.close()
        
        print(f"✓ HOLAPS trend plot saved for {station_name}")
        
    except Exception as e:
        print(f"Error processing {station_dir}: {str(e)}")
        import traceback
        print(traceback.format_exc())

print("All station directories processed!")

Found 48 station directories
Processing AT-Neu...
  P-values shape: (1, 15)
  Slopes shape: (1, 15)
  P-values columns: ['Analysis', 'Period', 'GHF', 'H', 'LE', 'Rn', 'Ta', 'Td', 'Ts', 'rain', 'sm1', 'sm2', 'sm3', 'sm4', 'sm5']
  Slopes columns: ['Analysis', 'Period', 'GHF', 'H', 'LE', 'Rn', 'Ta', 'Td', 'Ts', 'rain', 'sm1', 'sm2', 'sm3', 'sm4', 'sm5']
  Periods: ['2001-2020 (20)']
  Analyses: ['Complete Period']
    GHF: slope=0.0002, p-value=0.9741
    H: slope=0.0088, p-value=0.6265
    LE: slope=0.0136, p-value=0.8203
    Rn: slope=0.0336, p-value=0.6732
    Ta: slope=0.0636, p-value=0.0150
    Td: slope=0.0679, p-value=0.0179
    Ts: slope=0.0683, p-value=0.0179
    rain: slope=-0.0004, p-value=0.8203
    sm1: slope=-0.0002, p-value=0.3145
✓ HOLAPS trend plot saved for AT-Neu
Processing BE-Bra...
  P-values shape: (1, 11)
  Slopes shape: (1, 11)
  P-values columns: ['Analysis', 'Period', 'GHF', 'H', 'LE', 'Rn', 'Ta', 'Td', 'Ts', 'rain', 'sm1']
  Slopes columns: ['Analysis', 'Period

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.lines import Line2D

# Define HOLAPS parameters and assign fixed colors
color_dict = {
    'GHF': '#1f77b4',          # blue
    'H': '#ff7f0e',            # orange
    'LE': '#2ca02c',           # green
    'Rn': '#d62728',           # red
    'Ta': '#9467bd',           # purple
    'Td': '#8c564b',           # brown
    'Ts': '#e377c2',           # pink
    'rain': '#17becf',         # teal
    'sm1': '#bcbd22',           # olive
    'sm2': '#ff69b4',    # hot pink
    'sm3': '#00ced1',    # dark turquoise
    'sm4': '#ffa500',    # bright orange
    'sm5': '#8a2be2'    # blue violet
}

parameters = list(color_dict.keys())

# Fixed absolute Y positions for each parameter
parameter_distances = {
    'GHF': 0.2,
    'H': 0.4,
    'LE': 0.6,
    'Rn': 0.8,
    'Ta': 1.0,
    'Td': 1.2,
    'Ts': 1.4,
    'rain': 1.6,
    'sm1': 1.8,
    'sm2': 2.0,
    'sm3': 2.2,
    'sm4': 2.4,
    'sm5': 2.6
}

# Directory containing station subdirectories
input_directory = r'C:\Deepak\HOLAPS\monthly\station_data2'
output_directory = r'C:\Deepak\HOLAPS\monthly\station_data2\plots4'

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Get list of all station subdirectories
station_dirs = [d for d in os.listdir(input_directory)
                if os.path.isdir(os.path.join(input_directory, d)) and not d.lower().startswith('plots') and not d.lower().startswith('yearly')]
print(f"Found {len(station_dirs)} station directories")

# Group stations - you can modify this to group as needed
# Example: Group first 10 stations together
station_groups = []
group_size = 10

for i in range(0, len(station_dirs), group_size):
    station_groups.append(station_dirs[i:i + group_size])

print(f"Created {len(station_groups)} groups of stations")

# Process each group of stations
for group_idx, station_group in enumerate(station_groups):
    try:
        print(f"\nProcessing group {group_idx + 1}: {station_group}")
        
        # Create figure - wider for multiple stations
        fig, ax = plt.subplots(1, 1, figsize=(25, 12))
        
        all_station_data = []
        
        # Process each station in the group
        for station_idx, station_dir in enumerate(station_group):
            station_path = os.path.join(input_directory, station_dir)
            
            # Look for yearly trend file in this station directory
            trend_files = [f for f in os.listdir(station_path) 
                          if f.endswith('_yearly.xlsx') and 'trend' in f.lower()]
            
            if not trend_files:
                print(f"  No yearly trend file found for {station_dir}")
                continue
                
            trend_file = trend_files[0]
            station_name = station_dir
            
            print(f"  Processing {station_name}...")
            
            # File path
            file_path = os.path.join(station_path, trend_file)
            
            # Read the Excel file
            p_values_df = pd.read_excel(file_path, sheet_name='P-values')
            slopes_df = pd.read_excel(file_path, sheet_name='Slopes')
            
            # Clean the data
            p_values_df = p_values_df.dropna(how='all')
            slopes_df = slopes_df.dropna(how='all')
            p_values_df = p_values_df.reset_index(drop=True)
            slopes_df = slopes_df.reset_index(drop=True)
            
            # Get periods and analyses (should be just one row for yearly data)
            periods = slopes_df['Period'].values
            analyses = slopes_df['Analysis'].values
            
            # For each parameter, plot for this station
            for param in parameters:
                if param not in slopes_df.columns or param not in p_values_df.columns:
                    continue
                    
                slope = slopes_df[param].iloc[0]  # First (and only) row for yearly data
                p_val = p_values_df[param].iloc[0]

                if pd.isna(p_val) or pd.isna(slope):
                    continue

                # X position: station index (spaced out)
                x_position = station_idx * 2  # Space stations 2 units apart
                
                # Y position: parameter position + small random offset
                base_distance = parameter_distances[param]
                y_position = base_distance if slope >= 0 else -base_distance
                
                # Small vertical offset to avoid perfect alignment
                #y_offset = np.random.uniform(-0.05, 0.05)
                #y_position += y_offset

                # Marker rules based on p-values
                if p_val < 0.1:
                    marker = 'o'   # full circle
                    marker_size = 250
                    fill_style = 'full'
                    edge_width = 3.0
                    alpha_val = 1.0
                elif p_val < 0.2:
                    marker = '^'   # triangle
                    marker_size = 220
                    fill_style = 'full'
                    edge_width = 3.0
                    alpha_val = 0.9
                else:
                    marker = 'o'   # open circle
                    marker_size = 200
                    fill_style = 'none'
                    edge_width = 2.5
                    alpha_val = 0.8

                # Plot marker
                ax.scatter(x_position, y_position, s=marker_size,
                           c=color_dict[param] if fill_style == 'full' else 'none',
                           marker=marker, edgecolors=color_dict[param],
                           linewidths=edge_width, alpha=alpha_val, zorder=5,
                           label=f'{station_name}_{param}' if station_idx == 0 else "")
        
        # Customize plot
        ax.set_xlabel('Stations', fontsize=20, fontweight='bold')
        ax.set_xticks(range(0, len(station_group) * 2, 2))
        ax.set_xticklabels(station_group, rotation=45, ha='right', fontsize=14)
        
        # Set y-axis limits based on maximum distance
        max_distance = max(parameter_distances.values())
        y_limit = max_distance * 1.3  # Add some padding
        ax.set_ylim(-y_limit, y_limit)
        
        # Remove y-axis labels and ticks
        ax.set_yticks([])
        ax.set_ylabel('')
        
        # Zero line
        ax.axhline(y=0, color='black', linestyle='-', alpha=0.8, linewidth=2, zorder=1)
        
        # Add vertical lines to separate stations
        for i in range(len(station_group) + 1):
            ax.axvline(x=i * 2 - 1, color='gray', linestyle='--', alpha=0.3, linewidth=1, zorder=1)

        # Add grid for better readability
        ax.grid(True, axis='y', linestyle='--', alpha=0.3, linewidth=1, zorder=1)

        # Main title
        plt.title(f'HOLAPS Yearly  Trend Analysis - Stations {group_idx * group_size + 1} to {group_idx * group_size + len(station_group)}',
                 fontsize=22, fontweight='bold', pad=20)

        # Customize spines
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['left'].set_visible(False)

        # Create legends
        legend_elements = [
            Line2D([0], [0], marker='s', color='w',
                   markerfacecolor=color_dict[param],
                   markersize=12, label=param)
            for param in parameters
        ]

        marker_elements = [
            Line2D([0], [0], marker='o', color='black', markerfacecolor='black',
                   markersize=15, linestyle='None', label='p < 0.1 (Full circle)'),
            Line2D([0], [0], marker='^', color='black', markerfacecolor='black',
                   markersize=15, linestyle='None', label='0.1 ≤ p < 0.2 (Triangle)'),
            Line2D([0], [0], marker='o', color='black', markerfacecolor='none',
                   markersize=15, linestyle='None', label='p ≥ 0.2 (Open circle)')
        ]

        # Place legends
        legend1 = ax.legend(handles=legend_elements, 
                          loc='upper left',
                          bbox_to_anchor=(1.02, 1),
                          fontsize=12,
                          title='HOLAPS Parameters', 
                          title_fontsize=14)
        
        ax.add_artist(legend1)

        legend2 = ax.legend(handles=marker_elements, 
                          loc='lower left',
                          bbox_to_anchor=(1.02, 0.35),
                          fontsize=12,
                          title='Significance Levels', 
                          title_fontsize=14)

        # Add both legends to the plot
       
        
        # Adjust layout for larger elements
        plt.tight_layout(rect=[0, 0, 0.85, 0.95])
        
        # Save the plot with high DPI
        output_path = os.path.join(output_directory, f"holaps_trend_group_{group_idx + 1}.png")
        plt.savefig(output_path, dpi=350, bbox_inches='tight')
        plt.close()
        
        print(f"✓ Group {group_idx + 1} plot saved with {len(station_group)} stations")
        
    except Exception as e:
        print(f"Error processing group {group_idx + 1}: {str(e)}")
        import traceback
        print(traceback.format_exc())

print("All station groups processed!")

Found 44 station directories
Created 5 groups of stations

Processing group 1: ['AT-Neu', 'BE-Bra', 'BE-Lon', 'BE-Vie', 'CH-Cha', 'CH-Dav', 'CH-Fru', 'CZ-BK1', 'CZ-wet', 'DE-Geb']
  Processing AT-Neu...
  Processing BE-Bra...
  Processing BE-Lon...
  Processing BE-Vie...
  Processing CH-Cha...
  Processing CH-Dav...
  Processing CH-Fru...
  Processing CZ-BK1...
  Processing CZ-wet...
  Processing DE-Geb...
✓ Group 1 plot saved with 10 stations

Processing group 2: ['DE-Gri', 'DE-Hai', 'DE-HoH', 'DE-Hzd', 'DE-Kli', 'DE-Lnf', 'DE-Obe', 'DE-RuR', 'DE-RuS', 'DE-Tha']
  Processing DE-Gri...
  Processing DE-Hai...
  Processing DE-HoH...
  Processing DE-Hzd...
  Processing DE-Kli...
  Processing DE-Lnf...
  Processing DE-Obe...
  Processing DE-RuR...
  Processing DE-RuS...
  Processing DE-Tha...
✓ Group 2 plot saved with 10 stations

Processing group 3: ['DK-Sor', 'ES-Agu', 'ES-LJu', 'FR-Aur', 'FR-Bil', 'FR-Gri', 'FR-Hes', 'FR-Lam', 'FR-LBr', 'IT-BCi']
  Processing DK-Sor...
  Processing ES-A

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib_venn import venn2, venn3
import seaborn as sns

# Define parameters for HOLAPS
parameters = [
    'GHF', 'H', 'LE', 'Rn', 'Ta', 'Td', 'Ts', 
    'rain', 'sm1', 'sm2', 'sm3', 'sm4', 'sm5'
]

# Directory containing Excel files
input_directory = r'C:\Deepak\HOLAPS\monthly\station_data2\yearly'
output_directory = r'C:\Deepak\HOLAPS\monthly\station_data2\yearly\plots3'

# Create output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Global storage for significance data across all stations
all_stations_selected_data = []

# Get list of Excel files - separate yearly and JJA files
yearly_files = [f for f in os.listdir(input_directory) if f.endswith('_yearly.xlsx')]
jja_files = [f for f in os.listdir(input_directory) if f.endswith('_JJA.xlsx')]

# Create a mapping between yearly and JJA files
file_pairs = {}
for y_file in yearly_files:
    base_name = y_file.replace('_yearly.xlsx', '')
    jja_file = base_name + '_JJA.xlsx'
    if jja_file in jja_files:
        file_pairs[base_name] = {'yearly': y_file, 'jja': jja_file}
    else:
        file_pairs[base_name] = {'yearly': y_file, 'jja': None}

# Also include JJA files that don't have yearly counterparts
for j_file in jja_files:
    base_name = j_file.replace('_JJA.xlsx', '')
    if base_name not in file_pairs:
        file_pairs[base_name] = {'yearly': None, 'jja': j_file}

def get_selected_period_data(station_name, p_values_df, slopes_df, parameters, season):
    """Get data for all periods (no weighting/selection needed for HOLAPS)"""
    selected_data = []
    
    # For HOLAPS, we use only the "Complete Period" row (first row)
    # Skip the "Single Variable" rows as they are redundant
    complete_period_mask = p_values_df['Analysis'] == 'Complete Period'
    
    if complete_period_mask.any():
        period_idx = complete_period_mask.idxmax()
        
        for param in parameters:
            if param not in p_values_df.columns:
                continue
                
            p_val = p_values_df[param].iloc[period_idx]
            
            # Handle slope data - check if parameter exists in slopes_df
            if param in slopes_df.columns:
                slope = slopes_df[param].iloc[period_idx]
            else:
                slope = np.nan
            
            # Skip if p-value or slope is NaN
            if pd.isna(p_val) or pd.isna(slope):
                continue
            
            # Determine significance level
            if p_val < 0.1:
                sig_level = 'high'
            elif p_val < 0.2:
                sig_level = 'medium'
            else:
                sig_level = 'insig'
            
            selected_data.append({
                'station': station_name,
                'parameter': param,
                'season': season,
                'significance': sig_level,
                'p_value': p_val,
                'slope': slope,
                'period': p_values_df['Period'].iloc[period_idx] if 'Period' in p_values_df.columns else 'Complete Period'
            })
    
    return selected_data


def create_comprehensive_bar_plots_separate_seasons():
    """Create separate comprehensive bar plots for Yearly and JJA seasons"""
    
    # Filter data by season
    yearly_data = [record for record in all_stations_selected_data if record['season'] == 'yearly']
    jja_data = [record for record in all_stations_selected_data if record['season'] == 'JJA']
    
    # Pre-calculate maximum bar height for consistent Y-axis
    max_bar_height = 0
    
    for season_data in [yearly_data, jja_data]:
        if len(season_data) == 0:
            continue
            
        param_summary = {}
        for param in parameters:
            param_summary[param] = {'high_sig': 0, 'medium_sig': 0, 'insig_sig': 0}
        
        for record in season_data:
            param = record['parameter']
            sig_level = record['significance']
            
            if sig_level == 'high':
                key = 'high_sig'
            elif sig_level == 'medium':
                key = 'medium_sig'
            else:
                key = 'insig_sig'
            
            param_summary[param][key] += 1
        
        # Calculate total height for each parameter
        for param in parameters:
            if param in param_summary:
                total = (param_summary[param]['high_sig'] + 
                        param_summary[param]['medium_sig'] + 
                        param_summary[param]['insig_sig'])
                max_bar_height = max(max_bar_height, total)
    
    # Add some padding to the max height
    y_max = max_bar_height * 1.15
    
    # Create separate plots for yearly and JJA
    for season, season_data, season_name in [('yearly', yearly_data, 'Yearly'), ('JJA', jja_data, 'JJA (Summer)')]:
        if len(season_data) == 0:
            print(f"No data available for {season_name}")
            continue
            
        # Create summary data for bar plot
        param_summary = {}
        for param in parameters:
            param_summary[param] = {'high_sig': 0, 'medium_sig': 0, 'insig_sig': 0}
        
        for record in season_data:
            param = record['parameter']
            sig_level = record['significance']
            
            if sig_level == 'high':
                key = 'high_sig'
            elif sig_level == 'medium':
                key = 'medium_sig'
            else:
                key = 'insig_sig'
            
            param_summary[param][key] += 1
        
        # Prepare data for stacked bar plot - REVERSED ORDER
        params = list(param_summary.keys())
        insig = [param_summary[param]['insig_sig'] for param in params]
        medium_sig = [param_summary[param]['medium_sig'] for param in params]
        high_sig = [param_summary[param]['high_sig'] for param in params]
        
        # Create the plot
        fig, ax = plt.subplots(figsize=(16, 10))
        
        bar_width = 0.8
        x_pos = np.arange(len(params))
        
        # Plot in reversed order: Not significant at bottom, then medium, then high
        bars1 = ax.bar(x_pos, insig, bar_width, label='Not Significant (p ≥ 0.2)', 
                       color='#d62728', edgecolor='black', linewidth=1)
        bars2 = ax.bar(x_pos, medium_sig, bar_width, bottom=insig, 
                       label='Moderately Significant (0.1 ≤ p < 0.2)', 
                       color='#ff7f0e', edgecolor='black', linewidth=1)
        bars3 = ax.bar(x_pos, high_sig, bar_width, bottom=np.array(insig) + np.array(medium_sig),
                       label='Highly Significant (p < 0.1)', 
                       color='#2ca02c', edgecolor='black', linewidth=1)
        
        # Add value labels on bars - FIXED POSITIONING
        for i, (n, m, h) in enumerate(zip(insig, medium_sig, high_sig)):
            total = n + m + h
            if total > 0:
                # Total count at top of bar - position relative to bar height
                y_pos_total = total + (y_max * 0.02)  # Small offset above bar
                ax.text(i, y_pos_total, f'{total}', ha='center', va='bottom', 
                       fontweight='bold', fontsize=10, color='black')
                
                # Individual counts inside segments - only if segment is large enough
                segment_threshold = y_max * 0.08  # Only label segments taller than 8% of max height
                
                if n >= segment_threshold:  # Not significant
                    ax.text(i, n/2, f'{n}', ha='center', va='center', 
                           fontweight='bold', fontsize=9, color='white')
                
                if m >= segment_threshold:  # Moderately significant
                    ax.text(i, n + m/2, f'{m}', ha='center', va='center', 
                           fontweight='bold', fontsize=9, color='white')
                
                if h >= segment_threshold:  # Highly significant
                    ax.text(i, n + m + h/2, f'{h}', ha='center', va='center', 
                           fontweight='bold', fontsize=9, color='white')
        
        # Set consistent Y-axis limits
        ax.set_ylim(0, y_max)
        
        # Customize plot
        ax.set_xlabel('Parameters', fontsize=14, fontweight='bold')
        ax.set_ylabel('Number of Stations', fontsize=14, fontweight='bold')
        ax.set_title(f'Significance Analysis - {season_name} Season (HOLAPS)', 
                     fontsize=16, fontweight='bold', pad=20)
        ax.set_xticks(x_pos)
        ax.set_xticklabels(params, rotation=45, ha='right', fontsize=12)
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=12)
        ax.grid(True, axis='y', alpha=0.3)
        
        # Calculate statistics for annotation
        total_stations = len(set([record['station'] for record in season_data]))
        total_high = sum(high_sig)
        total_medium = sum(medium_sig)
        total_insig = sum(insig)
        total_all = total_high + total_medium + total_insig
        sig_percentage = (total_high + total_medium) / total_all * 100 if total_all > 0 else 0
        
        # Add statistics outside the plot area
        stats_text = (
            f'Total Stations: {total_stations}\n'
            f'Total Parameters: {total_all}\n'
            f'Highly Significant: {total_high} ({total_high/total_all*100:.1f}%)\n'
            f'Moderately Significant: {total_medium} ({total_medium/total_all*100:.1f}%)\n'
            f'Not Significant: {total_insig} ({total_insig/total_all*100:.1f}%)\n'
            f'Overall Significant: {total_high + total_medium}/{total_all} ({sig_percentage:.1f}%)'
        )
        
        # Place statistics on the right side outside the plot
        ax.text(1.02, 0.5, stats_text, transform=ax.transAxes, fontsize=11, 
                fontweight='bold', verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
                linespacing=1.5)
        
        # Adjust plot margins to accommodate the statistics text
        plt.subplots_adjust(right=0.75)
        
        plt.tight_layout()
        plt.savefig(os.path.join(output_directory, f'comprehensive_significance_{season}.png'), 
                    dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"Comprehensive bar plot created for {season_name} season")

def create_venn_diagrams_separate_seasons():
    """Create separate Venn diagrams for Yearly and JJA seasons with requested combinations"""
    
    if len(all_stations_selected_data) == 0:
        print("No selected period data found for Venn diagrams")
        return
    
    # Create separate Venn diagrams for yearly and JJA
    for season, season_name in [('yearly', 'Yearly'), ('JJA', 'JJA (Summer)')]:
        # Filter data by season and significance
        season_data = [record for record in all_stations_selected_data if record['season'] == season]
        sig_season_data = [record for record in season_data if record['p_value'] < 0.2]
        
        if len(sig_season_data) == 0:
            print(f"No significant records found for {season_name} Venn diagrams")
            continue
        
        # Create DataFrame for easier manipulation
        sig_df = pd.DataFrame(sig_season_data)
        
        # Group by station and parameter to get stations with at least one significant parameter
        station_params = sig_df.groupby(['station', 'parameter']).size().reset_index()
        
        # Create sets for different parameter groups
        le_stations = set(station_params[station_params['parameter'] == 'LE']['station'])
        h_stations = set(station_params[station_params['parameter'] == 'H']['station'])
        rn_stations = set(station_params[station_params['parameter'] == 'Rn']['station'])
        sm1_stations = set(station_params[station_params['parameter'] == 'sm1']['station'])
        rain_stations = set(station_params[station_params['parameter'] == 'rain']['station'])
        ta_stations = set(station_params[station_params['parameter'] == 'Ta']['station'])
        ts_stations = set(station_params[station_params['parameter'] == 'Ts']['station'])
        ghf_stations = set(station_params[station_params['parameter'] == 'GHF']['station'])
        td_stations = set(station_params[station_params['parameter'] == 'Td']['station'])
        
        # For ALL combo - stations that have significant trends in ALL 13 parameters
        all_params_stations = set()
        all_param_sets = [le_stations, h_stations, rn_stations, 
                         ta_stations, td_stations, ts_stations,
                         rain_stations, sm1_stations, ghf_stations]
        
       
        
        # Create Venn diagrams - 2x2 grid for the 5 combinations
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 14))
        
        # Combo 1: LE vs H vs Rn
        if le_stations or h_stations or rn_stations:
            venn3([le_stations, h_stations, rn_stations],
                  ['LE', 'H', 'Rn'], ax=ax1)
            ax1.set_title(f'1. LE vs H vs Rn\n({season_name})', fontsize=14, fontweight='bold')
        else:
            ax1.text(0.5, 0.5, 'No significant data', ha='center', va='center', transform=ax1.transAxes, fontsize=12)
            ax1.set_title(f'1. LE vs H vs Rn\n({season_name})', fontsize=14, fontweight='bold')
        
        # Combo 2: LE vs sm1 vs rain
        if le_stations or sm1_stations or rain_stations:
            venn3([le_stations, sm1_stations, rain_stations],
                  ['LE', 'sm1', 'rain'], ax=ax2)
            ax2.set_title(f'2. LE vs sm1 vs rain\n({season_name})', fontsize=14, fontweight='bold')
        else:
            ax2.text(0.5, 0.5, 'No significant data', ha='center', va='center', transform=ax2.transAxes, fontsize=12)
            ax2.set_title(f'2. LE vs sm1 vs rain\n({season_name})', fontsize=14, fontweight='bold')
        
       
      
        # Combo 3: H vs Ta vs Ts
        if h_stations or ta_stations or ts_stations:
            venn3([h_stations, ta_stations, ts_stations],
                  ['H', 'Ta', 'Ts'], ax=ax3)
            ax3.set_title(f'3. H vs Ta vs Ts\n({season_name})', fontsize=14, fontweight='bold')
        else:
            ax3.text(0.5, 0.5, 'No significant data', ha='center', va='center', transform=ax3.transAxes, fontsize=12)
            ax3.set_title(f'3. H vs Ta vs Ts\n({season_name})', fontsize=14, fontweight='bold')

        # Combo 4: H vs GHF vs sm1
        if h_stations or ghf_stations or sm1_stations:
            venn3([h_stations, ghf_stations, sm1_stations],
                  ['H', 'GHF', 'sm1'], ax=ax4)
            ax4.set_title(f'4. H vs GHF vs sm1\n({season_name})', fontsize=14, fontweight='bold')
        else:
            ax4.text(0.5, 0.5, 'No significant data', ha='center', va='center', transform=ax4.transAxes, fontsize=12)
            ax4.set_title(f'4. H vs GHF vs sm1\n({season_name})', fontsize=14, fontweight='bold')
        
       
       
        print(f"Venn diagrams created for {season_name} season")    
        
        plt.suptitle(f'Venn Diagrams - {season_name} Season (HOLAPS)\n(p < 0.2)', 
                     fontsize=16, fontweight='bold', y=0.95)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.savefig(os.path.join(output_directory, f'venn_diagrams_{season}.png'), 
                    dpi=300, bbox_inches='tight')
      
     
        plt.close()
        
        print(f"Venn diagrams created for {season_name} season")

# [Keep the save_comprehensive_selection_file() function as before, just rename the output file]
def save_comprehensive_selection_file():
    """Save comprehensive data to Excel file with separate sheets for seasons"""
    if len(all_stations_selected_data) == 0:
        print("No data to save")
        return
    
    # Convert to DataFrame
    comp_df = pd.DataFrame(all_stations_selected_data)
    
    # Save to Excel
    output_file = os.path.join(output_directory, 'comprehensive_holaps_analysis.xlsx')
    
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # Save main data
        comp_df.to_excel(writer, sheet_name='All_Data', index=False)
        
        # Create separate sheets for yearly and JJA
        for season, season_name in [('yearly', 'Yearly'), ('JJA', 'JJA_Summer')]:
            season_data = comp_df[comp_df['season'] == season]
            if len(season_data) > 0:
                season_data.to_excel(writer, sheet_name=f'{season_name}_Data', index=False)
        
        # Create summary sheet
        summary_data = []
        stations = comp_df['station'].unique()
        
        for station in stations:
            station_data = comp_df[comp_df['station'] == station]
            # Separate by season
            yearly_data = station_data[station_data['season'] == 'yearly']
            jja_data = station_data[station_data['season'] == 'JJA']
            
            # Yearly summary
            if len(yearly_data) > 0:
                high_sig_yearly = len(yearly_data[yearly_data['significance'] == 'high'])
                medium_sig_yearly = len(yearly_data[yearly_data['significance'] == 'medium'])
                insig_yearly = len(yearly_data[yearly_data['significance'] == 'insig'])
                total_params_yearly = len(yearly_data)
            else:
                high_sig_yearly = medium_sig_yearly = insig_yearly = total_params_yearly = 0
            
            # JJA summary
            if len(jja_data) > 0:
                high_sig_jja = len(jja_data[jja_data['significance'] == 'high'])
                medium_sig_jja = len(jja_data[jja_data['significance'] == 'medium'])
                insig_jja = len(jja_data[jja_data['significance'] == 'insig'])
                total_params_jja = len(jja_data)
            else:
                high_sig_jja = medium_sig_jja = insig_jja = total_params_jja = 0
            
            summary_data.append({
                'Station': station,
                'Yearly_Total_Params': total_params_yearly,
                'Yearly_High_Sig': high_sig_yearly,
                'Yearly_Medium_Sig': medium_sig_yearly,
                'Yearly_Not_Sig': insig_yearly,
                'JJA_Total_Params': total_params_jja,
                'JJA_High_Sig': high_sig_jja,
                'JJA_Medium_Sig': medium_sig_jja,
                'JJA_Not_Sig': insig_jja
            })
        
        summary_df = pd.DataFrame(summary_data)
        summary_df.to_excel(writer, sheet_name='Station_Summary', index=False)
        
        # Create parameter summary sheets for each season
        for season, season_name in [('yearly', 'Yearly'), ('JJA', 'JJA')]:
            season_data = comp_df[comp_df['season'] == season]
            if len(season_data) > 0:
                param_summary_data = []
                for param in parameters:
                    param_data = season_data[season_data['parameter'] == param]
                    total_stations = len(param_data)
                    high_sig = len(param_data[param_data['significance'] == 'high'])
                    medium_sig = len(param_data[param_data['significance'] == 'medium'])
                    insig = len(param_data[param_data['significance'] == 'insig'])
                    
                    param_summary_data.append({
                        'Parameter': param,
                        'Total_Stations': total_stations,
                        'Highly_Significant': high_sig,
                        'Moderately_Significant': medium_sig,
                        'Not_Significant': insig,
                        'Significant_Count': high_sig + medium_sig,
                        'Significant_Percentage': ((high_sig + medium_sig) / total_stations * 100) if total_stations > 0 else 0
                    })
                
                param_summary_df = pd.DataFrame(param_summary_data)
                param_summary_df.to_excel(writer, sheet_name=f'Parameter_Summary_{season_name}', index=False)
    
    print(f"Comprehensive HOLAPS file saved: {output_file}")

# Main processing - collect data from all stations
print("Collecting data from all HOLAPS stations...")

for base_name, files in file_pairs.items():
    try:
        # Process yearly data
        if files['yearly']:
            yearly_file_path = os.path.join(input_directory, files['yearly'])
            try:
                p_values_df_yearly = pd.read_excel(yearly_file_path, sheet_name='P-values')
                slopes_df_yearly = pd.read_excel(yearly_file_path, sheet_name='Slopes')
                
                # Clean the data - only use "Complete Period" row
                p_values_df_yearly = p_values_df_yearly.dropna(how='all')
                slopes_df_yearly = slopes_df_yearly.dropna(how='all')
                
                # Store only the "Complete Period" data for HOLAPS
                selected_data = get_selected_period_data(
                    base_name, p_values_df_yearly, slopes_df_yearly, 
                    parameters, 'yearly')
                all_stations_selected_data.extend(selected_data)
                print(f"Collected yearly data for {base_name} - {len(selected_data)} parameters")
                
            except Exception as e:
                print(f"Error loading yearly data for {base_name}: {str(e)}")
        
        # Process JJA data
        if files['jja']:
            jja_file_path = os.path.join(input_directory, files['jja'])
            try:
                p_values_df_jja = pd.read_excel(jja_file_path, sheet_name='P-values')
                slopes_df_jja = pd.read_excel(jja_file_path, sheet_name='Slopes')
                
                # Clean the data - only use "Complete Period" row
                p_values_df_jja = p_values_df_jja.dropna(how='all')
                slopes_df_jja = slopes_df_jja.dropna(how='all')
                
                # Store only the "Complete Period" data for HOLAPS
                selected_data = get_selected_period_data(
                    base_name, p_values_df_jja, slopes_df_jja, 
                    parameters, 'JJA')
                all_stations_selected_data.extend(selected_data)
                print(f"Collected JJA data for {base_name} - {len(selected_data)} parameters")
                
            except Exception as e:
                print(f"Error loading JJA data for {base_name}: {str(e)}")
                
    except Exception as e:
        print(f"Error processing {base_name}: {str(e)}")

print(f"Data collection complete. Total records: {len(all_stations_selected_data)}")



# Create comprehensive plots and save file
print("Creating comprehensive analysis plots...")

# Save comprehensive selection file
save_comprehensive_selection_file()

# Create comprehensive bar plots (separate for yearly and JJA)
create_comprehensive_bar_plots_separate_seasons()
print("Comprehensive bar plots created for both seasons!")

# Create Venn diagrams (separate for yearly and JJA)
create_venn_diagrams_separate_seasons()
print("Venn diagrams created for both seasons!")

print("All comprehensive HOLAPS analysis completed!")

Collected yearly data for holaps_trend_tables_AT-Neu - 13 parameters
Collected JJA data for holaps_trend_tables_AT-Neu - 13 parameters
Collected yearly data for holaps_trend_tables_BE-Bra - 13 parameters
Collected JJA data for holaps_trend_tables_BE-Bra - 13 parameters
Collected yearly data for holaps_trend_tables_BE-Lon - 13 parameters
Collected JJA data for holaps_trend_tables_BE-Lon - 13 parameters
Collected yearly data for holaps_trend_tables_BE-Vie - 13 parameters
Collected JJA data for holaps_trend_tables_BE-Vie - 13 parameters
Collected yearly data for holaps_trend_tables_CH-Cha - 13 parameters
Collected JJA data for holaps_trend_tables_CH-Cha - 13 parameters
Collected yearly data for holaps_trend_tables_CH-Dav - 13 parameters
Collected JJA data for holaps_trend_tables_CH-Dav - 13 parameters
Collected yearly data for holaps_trend_tables_CH-Fru - 13 parameters
Collected JJA data for holaps_trend_tables_CH-Fru - 13 parameters
Collected yearly data for holaps_trend_tables_CZ-BK1 - 

c:\Users\chinthap\AppData\Local\miniconda3\envs\new\Lib\site-packages\matplotlib_venn\layout\venn3\pairwise.py:107: UserWarning: Circle B has zero area.
  warnings.warn("Circle B has zero area.")


Venn diagrams created for Yearly season
Venn diagrams created for JJA (Summer) season
Venn diagrams created for JJA (Summer) season
Venn diagrams created for both seasons!
All comprehensive HOLAPS analysis completed!
